In [1]:
from typing import List, Tuple, Union, Optional, Callable, Any

# Define a type alias for numeric values
Numeric = Union[int, float]

try:
    from scipy.sparse import spmatrix
except ImportError:
    spmatrix = Any  # Fallback if SciPy is not installed

class MatrixConverter:
    """
    A class for converting between dense and sparse matrix representations.

    The dense matrix is represented as a list of lists of numbers.
    The sparse representation can be:
      - A native Python representation: a list of tuples (row_index, col_index, value)
        for each nonzero entry.
      - An external SciPy sparse matrix (if using NumPy/Scipy for conversion).

    This class provides:
      - Robust input validation and error handling.
      - Optional thresholding to treat near-zero values as zero.
      - Caching of conversion results.
      - Bidirectional conversion (dense ⇄ sparse).
    """

    def __init__(self, matrix: List[List[Numeric]]) -> None:
        self._validate_matrix(matrix)
        self.matrix: List[List[Numeric]] = matrix
        self.num_rows: int = len(matrix)
        self.num_cols: int = len(matrix[0])
        # Cache conversion results keyed by parameters
        self._cache: dict = {}

    @property
    def shape(self) -> Tuple[int, int]:
        """Returns the (rows, columns) shape of the dense matrix."""
        return (self.num_rows, self.num_cols)

    def __repr__(self) -> str:
        return f"MatrixConverter(shape={self.shape})"

    def _validate_matrix(self, matrix: List[List[Numeric]]) -> None:
        """Validates that the matrix is non-empty, rectangular, and numeric."""
        if not matrix or not isinstance(matrix, list):
            raise ValueError("Input matrix must be a non-empty list of lists.")
        if any(not isinstance(row, list) for row in matrix):
            raise ValueError("Input matrix must be a list of lists.")

        expected_length = len(matrix[0])
        for row in matrix:
            if len(row) != expected_length:
                raise ValueError("All rows in the matrix must have the same number of columns.")
            for item in row:
                if not isinstance(item, (int, float)):
                    raise ValueError("Matrix elements must be numeric (int or float).")

    def update_matrix(self, matrix: List[List[Numeric]]) -> None:
        """
        Updates the dense matrix and clears any cached conversion results.

        Parameters:
            matrix (List[List[Numeric]]): The new dense matrix.
        """
        self._validate_matrix(matrix)
        self.matrix = matrix
        self.num_rows = len(matrix)
        self.num_cols = len(matrix[0])
        self._cache.clear()

    def clear_cache(self) -> None:
        """Clears the cached conversion results."""
        self._cache.clear()

    def dense_to_sparse(
        self,
        use_external: bool = False,
        sparse_type: str = 'csr',
        threshold: Optional[Numeric] = None,
        custom_factory: Optional[Callable[[List[List[Numeric]]], Any]] = None
    ) -> Union[List[Tuple[int, int, Numeric]], Any]:
        """
        Converts the stored dense matrix into a sparse representation.

        When `use_external` is False (default), returns a native sparse representation as a list of
        tuples (row, col, value) for each nonzero element. If `threshold` is provided, values with
        absolute value below the threshold are treated as zero.

        When `use_external` is True, leverages NumPy and SciPy to convert the dense matrix into a
        SciPy sparse matrix. The `sparse_type` parameter specifies the format and supports:
        'csr', 'csc', 'coo', 'lil', 'dok'. Alternatively, a custom factory function can be provided
        to override the default external conversion.

        Parameters:
            use_external (bool): Whether to use external libraries for conversion (default is False).
            sparse_type (str): The SciPy sparse matrix format (default 'csr').
            threshold (Optional[Numeric]): Threshold below which values are considered zero.
            custom_factory (Optional[Callable]): Function that takes the dense matrix and returns a custom sparse representation.

        Returns:
            Union[List[Tuple[int, int, Numeric]], SciPy sparse matrix]:
                - Native sparse representation as a list of tuples.
                - External representation as a SciPy sparse matrix.

        Raises:
            ValueError: If an unsupported sparse_type is provided.
            ImportError: If external conversion is requested but NumPy/Scipy are not installed.
        """
        # Use caching when no custom factory is provided
        cache_key = (use_external, sparse_type.lower(), threshold) if custom_factory is None else None
        if cache_key is not None and cache_key in self._cache:
            return self._cache[cache_key]

        def is_nonzero(value: Numeric) -> bool:
            return abs(value) >= threshold if threshold is not None else value != 0

        if use_external:
            try:
                import numpy as np
                from scipy.sparse import csr_matrix, csc_matrix, coo_matrix, lil_matrix, dok_matrix
            except ImportError as e:
                raise ImportError("NumPy and SciPy must be installed for external conversion.") from e

            np_dense = np.array(self.matrix)
            if custom_factory:
                result = custom_factory(self.matrix)
            else:
                sparse_type_lower = sparse_type.lower()
                if sparse_type_lower == 'csr':
                    result = csr_matrix(np_dense)
                elif sparse_type_lower == 'csc':
                    result = csc_matrix(np_dense)
                elif sparse_type_lower == 'coo':
                    result = coo_matrix(np_dense)
                elif sparse_type_lower == 'lil':
                    result = lil_matrix(np_dense)
                elif sparse_type_lower == 'dok':
                    result = dok_matrix(np_dense)
                else:
                    raise ValueError("Unsupported sparse_type. Supported types: 'csr', 'csc', 'coo', 'lil', 'dok'.")
        else:
            result: List[Tuple[int, int, Numeric]] = []
            for i, row in enumerate(self.matrix):
                for j, value in enumerate(row):
                    if is_nonzero(value):
                        result.append((i, j, value))

        if cache_key is not None:
            self._cache[cache_key] = result
        return result

    def sparse_to_dense(
        self,
        sparse: Union[List[Tuple[int, int, Numeric]], Any]
    ) -> List[List[Numeric]]:
        """
        Converts a sparse representation back into a dense matrix.

        The sparse representation can be:
            - A native representation: a list of (row, col, value) tuples.
            - A SciPy sparse matrix (which supports the toarray() method).

        For the native representation, if the shape is not fully determined by the sparse data,
        the conversion uses the shape of the originally stored matrix.

        Parameters:
            sparse (Union[List[Tuple[int, int, Numeric]], SciPy sparse matrix]):
                The sparse representation to be converted.

        Returns:
            List[List[Numeric]]: The dense matrix as a list of lists.

        Raises:
            TypeError: If the sparse representation format is unsupported.
        """
        if hasattr(sparse, "toarray") and callable(sparse.toarray):
            # External conversion: use the SciPy sparse matrix's toarray() method
            return sparse.toarray().tolist()
        elif isinstance(sparse, list):
            # Determine the maximum indices from the sparse data
            max_row, max_col = self.num_rows - 1, self.num_cols - 1
            for i, j, _ in sparse:
                if i > max_row:
                    max_row = i
                if j > max_col:
                    max_col = j
            dense = [[0 for _ in range(max_col + 1)] for _ in range(max_row + 1)]
            for i, j, value in sparse:
                dense[i][j] = value
            return dense
        else:
            raise TypeError("Unsupported sparse representation format for dense conversion.")

    def density(
        self,
        sparse: Optional[Union[List[Tuple[int, int, Numeric]], Any]] = None
    ) -> float:
        """
        Computes the density of the matrix as the ratio of nonzero elements to total elements.

        If a sparse representation is provided, the density is computed based on that.
        Otherwise, the density is computed from the stored dense matrix.

        Parameters:
            sparse (Optional[Union[List[Tuple[int, int, Numeric]], SciPy sparse matrix]]):
                An optional sparse representation of the matrix.

        Returns:
            float: The density of the matrix.
        """
        if sparse is None:
            total_elements = self.num_rows * self.num_cols
            nonzero = sum(1 for row in self.matrix for value in row if value != 0)
        elif hasattr(sparse, "toarray") and callable(sparse.toarray):
            arr = sparse.toarray()
            total_elements = arr.size
            nonzero = int((arr != 0).sum())
        elif isinstance(sparse, list):
            nonzero = len(sparse)
            max_row, max_col = self.num_rows - 1, self.num_cols - 1
            for i, j, _ in sparse:
                if i > max_row:
                    max_row = i
                if j > max_col:
                    max_col = j
            total_elements = (max_row + 1) * (max_col + 1)
        else:
            raise TypeError("Unsupported sparse representation format for density calculation.")
        return nonzero / total_elements if total_elements > 0 else 0.0

In [2]:
if __name__ == "__main__":
    # Define an example dense matrix
    dense_matrix: List[List[Numeric]] = [
        [0, 0, 3.0, 0],
        [4.0, 0, 0, 2.0],
        [0, 0, 0, 0],
        [0, 5.0, 0, 0]
    ]

    # Create a MatrixConverter instance
    converter = MatrixConverter(dense_matrix)
    print("MatrixConverter instance:", converter)

    # --- Native Conversion with Threshold ---
    native_sparse = converter.dense_to_sparse(use_external=False, threshold=0.1)
    print("\nNative sparse representation:")
    print(native_sparse)

    # Convert native sparse back to dense
    recovered_dense_native = converter.sparse_to_dense(native_sparse)
    print("\nRecovered dense matrix (native):")
    for row in recovered_dense_native:
        print(row)

    # --- External Conversion using SciPy CSR Format ---
    try:
        external_sparse = converter.dense_to_sparse(use_external=True, sparse_type='csr')
        print("\nExternal sparse representation (CSR):")
        print(external_sparse)
        recovered_dense_external = converter.sparse_to_dense(external_sparse)
        print("\nRecovered dense matrix (external):")
        for row in recovered_dense_external:
            print(row)
    except ImportError as e:
        print("External conversion failed:", e)

    # --- Custom Factory Example ---
    def custom_factory(matrix: List[List[Numeric]]):
        # Create a custom sparse representation as a dictionary mapping (row, col) to value
        return {(i, j): value for i, row in enumerate(matrix) for j, value in enumerate(row) if value != 0}

    custom_sparse = converter.dense_to_sparse(use_external=True, custom_factory=custom_factory)
    print("\nCustom sparse representation (dictionary):")
    print(custom_sparse)

    # --- Density Calculation ---
    density_native = converter.density(native_sparse)
    print("\nDensity (native sparse):", density_native)

    density_dense = converter.density()
    print("Density (dense matrix):", density_dense)

MatrixConverter instance: MatrixConverter(shape=(4, 4))

Native sparse representation:
[(0, 2, 3.0), (1, 0, 4.0), (1, 3, 2.0), (3, 1, 5.0)]

Recovered dense matrix (native):
[0, 0, 3.0, 0]
[4.0, 0, 0, 2.0]
[0, 0, 0, 0]
[0, 5.0, 0, 0]

External sparse representation (CSR):
  (0, 2)	3.0
  (1, 0)	4.0
  (1, 3)	2.0
  (3, 1)	5.0

Recovered dense matrix (external):
[0.0, 0.0, 3.0, 0.0]
[4.0, 0.0, 0.0, 2.0]
[0.0, 0.0, 0.0, 0.0]
[0.0, 5.0, 0.0, 0.0]

Custom sparse representation (dictionary):
{(0, 2): 3.0, (1, 0): 4.0, (1, 3): 2.0, (3, 1): 5.0}

Density (native sparse): 0.25
Density (dense matrix): 0.25
